In [3]:
import os 
import numpy as np
import torchvision
from PIL import Image 
import torch

```
[envmap rotate] [face 10]
[Black image]   [Ball]
[envmap rotate] [face 10]
[Black image]   [Ball]
[envmap rotate] [face 10]
[Black image]   [Ball]
[envmap rotate] [face 10]
[Black image]   [Ball]
[envmap rotate] [face 10]
```

In [2]:
ROOT_DIR = "../../output/20240703/val_rotate_envmap/vae5000/3e-4/chk179"
IMAGE_TYPE = ['light_x_minus','light_x_plus','light_y_minus', 'light_y_plus','light_z_minus', 'light_z_plus']
AXIS = ['x','y','z']

In [33]:
import torch
from torchvision import transforms

def convert_to_grayscale_and_highlight(image):
  """
  Converts an image to grayscale, finds the average brightness,
  and sets pixels with matching grayscale values to red.

  Args:
      image: A PyTorch tensor of shape [3, 256, 256] representing an RGB image.

  Returns:
      A PyTorch tensor of shape [3, 256, 256] with the modified image.
  """

  # Convert to grayscale using weighted average (BT.709 standard)
  grayscale_transform = transforms.Grayscale(num_output_channels=1)
  grayscale_image = grayscale_transform(image)

  # Calculate average brightness
  brightness = grayscale_image.max()

  # Create a mask to select pixels with matching grayscale value
  mask = (grayscale_image > brightness).float()

  # Convert mask to a 3-channel tensor for element-wise multiplication
  mask = mask.expand_as(image)

  # Set matching pixels to red (assuming original image has uint8 dtype)
  #red_channel = torch.zeros_like(image)  # Create a zero red channel
  #red_channel[0] = 255
  #red_channel.fill_(255)  # Fill with maximum red intensity (255 for uint8)
  #modified_image = image * (1 - mask) + red_channel * mask
  modified_image = image.copy()
  for i in range(256):
    for j in range(256):
      print(grayscale_image[0,i,j], brightness)
      if grayscale_image[0,i,j] >= brightness:
        print("SET")
        modified_image[0,i,j] = 1.0
        modified_image[1,i,j] = 0.0
        modified_image[2,i,j] = 0.0
#   modified_image[0,grayscale_image >= brightness] = 1.0
#   modified_image[1,grayscale_image >= brightness] = 0.0
#   modified_image[2,grayscale_image >= brightness] = 0.0

  return modified_image

In [37]:
AXIS[1]

'y'

In [38]:
image_type = IMAGE_TYPE[0]
axis = AXIS[2]
current_dir = f"{ROOT_DIR}/{image_type}/lightning_logs/version_0/"
files = sorted(os.listdir(current_dir))
files = [f for f in files if os.path.exists(os.path.join(current_dir,f,axis+'_ball','047.png'))][:30]
for frame_id in range(48):
    # create a row of envmap and images 
    rows = []
    print("frame_id: ", frame_id)
    for row_id in range(3):
        # image row
        row = []
        scene_id = int(files[row_id*10].split("_")[0])
        # first add envmap picture 
        envmap = Image.open(f"{current_dir}/{scene_id:05}_00000/{axis}_env/{frame_id:03d}.png").resize((256,256))
        #convert envmap to tensor
        envmap = torchvision.transforms.ToTensor()(envmap)
        row.append(envmap)

        # for on 10 images 
        for image_id in range(0,10):
            image = Image.open(f"{current_dir}/{scene_id:05}_{image_id:05}/{axis}/{frame_id:03d}.png").resize((256,256))
            image = torchvision.transforms.ToTensor()(image)
            row.append(image)
        # concatenate row
        rows += row
        # ball row
        row = []
        #first ball is blank image
        row.append(torch.zeros(3, 256, 256).to(torch.uint8))
        for image_id in range(0,10):
            try:
                image = Image.open(f"{current_dir}/{scene_id:05}_{image_id:05}/{axis}_ball/{frame_id:03d}.png").resize((256,256))
                image = torchvision.transforms.ToTensor()(image)
                #image = convert_to_grayscale_and_highlight(image)
            except:
                image = torch.zeros(3, 256, 256).to(torch.uint8)
            row.append(image)
        rows += row
    # makegrid from rows with 11 columns per row 
    frame_image = torchvision.utils.make_grid(rows,nrow=11)
    # save image
    output_dir = f"{current_dir}/../frames_{axis}"
    os.makedirs(output_dir,exist_ok=True)
    torchvision.utils.save_image(frame_image,f"{output_dir}/{frame_id:03d}.jpg")



frame_id:  0
frame_id:  1
frame_id:  2
frame_id:  3
frame_id:  4
frame_id:  5
frame_id:  6
frame_id:  7
frame_id:  8
frame_id:  9
frame_id:  10
frame_id:  11
frame_id:  12
